In [1]:
import keras
import numpy as np
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from skimage.transform import resize

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

Using TensorFlow backend.


1 Physical GPUs, 1 Logical GPUs


In [2]:
NUM_CLASSES = 10
BATCH_SIZE = 5
NUM_EPOCHS = 10
use_data_aug = False

In [3]:
from tqdm import tqdm
# img_arr is of shape (n, h, w, c)
def resize_image_arr(img_arr):
    x_resized_list = []
    for i in tqdm(range(img_arr.shape[0])):
        img = img_arr[0]
        resized_img = resize(img, (224, 224))
        x_resized_list.append(resized_img)
    return np.stack(x_resized_list)

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
x_train = x_train[0:10000]
y_train = y_train[0:10000]
x_test = x_test[0:3]
y_test = y_test[0:3]

In [21]:
x_train.shape

(10000, 32, 32, 3)

In [6]:
# Resize image arrays
x_train = resize_image_arr(x_train)
x_test = resize_image_arr(x_test)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 44.96it/s]


In [7]:

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [8]:
# Normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [9]:

base_model = ResNet50(include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
# and a logistic layer -- 10 classes for CIFAR10
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0, decay=0.0)

# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=2,
          epochs=NUM_EPOCHS,
          validation_data=(x_test, y_test),
          shuffle=False)

C:\Users\benza-ord\anaconda3\envs\keras-gpu\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Train on 10000 samples, validate on 3 samples
Epoch 1/10
10000/10000 [==============================] - 662s 66ms/step - loss: 0.3293 - accuracy: 0.9000 - val_loss: 0.3866 - val_accuracy: 0.9000
Epoch 2/10
10000/10000 [==============================] - 650s 65ms/step - loss: 0.3293 - accuracy: 0.9000 - val_loss: 0.3866 - val_accuracy: 0.9000
Epoch 3/10
10000/10000 [==============================] - 639s 64ms/step - loss: 0.3293 - accuracy: 0.9000 - val_loss: 0.3866 - val_accuracy: 0.9000
Epoch 4/10
10000/10000 [==============================] - 636s 64ms/step - loss: 0.3293 - accuracy: 0.9000 - val_loss: 0.3866 - val_accuracy: 0.9000
Epoch 5/10
10000/10000 [==============================] - 635s 64ms/step - loss: 0.3293 - accuracy: 0.9000 - val_loss: 0.3866 - val_accuracy: 0.9000
Epoch 6/10
10000/10000 [==============================] - 653s 65ms/step - loss: 0.3293 - accuracy: 0.9000 - val_loss: 0.3866 - val_accuracy: 0.9000
Epoch 7/10
10000/10000 [==============================] - 65

In [11]:
model.save('resnet_model_v1')
